In [1]:
# Load env in local
from dotenv import load_dotenv
import os
import getpass

load_dotenv()

if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

os.environ["LANGSMITH_TRACING_V2"] = "true"
if not os.getenv("LANGSMITH_API_KEY"):
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
    
print("Loaded OPENAI & LANGSMITH variables")

Loaded OPENAI & LANGSMITH variables


In [2]:
# Load vectordb
import faiss
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
)

index = faiss.read_index("../docs/faiss/faiss_index.index")
print("Loaded existing vector store.")

Loaded existing vector store.


In [3]:
# Load LLM
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [4]:
# Init graph_builder
from langgraph.graph import MessagesState, StateGraph
graph_builder = StateGraph(MessagesState)

In [5]:
from langgraph.graph import MessagesState, StateGraph
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import ToolNode
from langchain_core.tools import tool
import numpy as np
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Tool retrieve define
# @tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    loader = PyPDFLoader("../knowledge_pdf/masterigrandview.pdf")
    pages = loader.load()

    # 2. Splitter
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=500,
        chunk_overlap=70,
        length_function=len
    )
    
    docs = text_splitter.split_documents(pages)
    print(query)
    # Chuyển câu truy vấn thành vector embedding
    query_vector = embeddings.embed_query(query)
 
    # Tìm kiếm trong chỉ mục FAISS
    D, I = index.search(np.array([query_vector], dtype=np.float32), k=2)
    retrieved_docs = []
    
    for idx in I[0]:
        retrieved_docs.append(docs[idx])
    
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
        for doc in retrieved_docs
    )
    print(retrieved_docs)
    return serialized, retrieved_docs

# tools = ToolNode([retrieve])

# Step 2: Generate a response using the retrieved content.
def generate(state: MessagesState):
    """Generate answer."""
    loader = PyPDFLoader("../knowledge_pdf/masterigrandview.pdf")
    pages = loader.load()

    # 2. Splitter
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=500,
        chunk_overlap=70,
        length_function=len
    )
    
    docs = text_splitter.split_documents(pages)
    # Chuyển câu truy vấn thành vector embedding
    query = state["messages"][-1].content
    query_vector = embeddings.embed_query(query)
 
    # Tìm kiếm trong chỉ mục FAISS
    D, I = index.search(np.array([query_vector], dtype=np.float32), k=5)
    retrieved_docs = []
    
    for idx in I[0]:
        retrieved_docs.append(docs[idx])
    
    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    system_message_content = (
        "Bạn là nhân viên chăm sóc khách hàng của dự án Masteri Grand View và bạn sẽ trả lời các câu hỏi của khách hàng về dự án Masteri Grand View."
        "Hãy dùng đại từ xưng hô gọi khách hàng là Anh/Chị, còn bạn dùng đại từ xưng hô là Em."
        "Hãy trả lời câu hỏi của khách hàng một cách lịch sự và tôn trọng."
        "Chỉ dựa trên thông tin trong tài liệu dự án Masteri Grand View bên dưới, trả lời câu hỏi:"
        "\n\n"
        f"{docs_content}"
        "\n\n"
        "Trong trường hợp khách hàng hỏi những câu hỏi không liên quan đến Công ty và dự án, hãy từ chối trả lời một cách lịch sự."
        "Nếu thông tin không có trong tài liệu, hãy trả lời với khách hàng rằng thông tin này bạn không rõ và nói khách hàng gọi cho bộ phận kinh doanh"
        "Không bao giờ sáng tạo nội dung"
    )
    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system", "ai")
    ]
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # Run
    response = llm.invoke(prompt)
    return {"messages": [response]}

In [6]:
from langgraph.graph import END, START
from langgraph.prebuilt import tools_condition
from langgraph.checkpoint.memory import MemorySaver

# Graph builder
# graph_builder.add_node(query_or_respond)
# graph_builder.add_node(retrieve)
graph_builder.add_node(generate)

# graph_builder.set_entry_point("tools")
# graph_builder.add_conditional_edges(
#     "query_or_respond",
#     tools_condition,
#     {END: END, "tools": "tools"},
# )
graph_builder.add_edge(START, "generate")
graph_builder.add_edge("generate", END)
# memory = MemorySaver()
# graph = graph_builder.compile(checkpointer=memory)
graph = graph_builder.compile()

In [7]:
query = "Dự án bên em có gì đặc biệt"
# Execute
config = {"configurable": {"thread_id": "abc123"}}
messages = [
    {"role": "user", "content": query}
]
response = await graph.ainvoke({"messages": messages}, config)
print(response["messages"][-1].content)

Dạ, dự án Masteri Grand View có nhiều điểm đặc biệt nổi bật. Đầu tiên, đây là một siêu phẩm cao tầng thuộc Masteri Collection, tọa lạc ngay tại trung tâm mới. Sảnh thang máy được thiết kế đẳng cấp, lấy cảm hứng từ vẻ đẹp thanh nguyên của chất liệu và đường nét, tạo nên dấu ấn rõ nét cho dự án.

Ngoài ra, ban công rộng rãi với lan can kính cao 1,4m giúp tạo view nhìn thoáng đãng. Mặt bằng thiết kế cho phép gia chủ tối ưu thi công thêm phần đảo bếp nếu muốn. Logia được bố trí khéo léo để tạo không gian cho sân phơi, cùng với hệ lam kín đáo che chắn, tăng tính thẩm mỹ cho toàn bộ dự án.

Tất cả các phòng ngủ đều được thiết kế với hệ cửa kính full trần đến sàn, tối ưu hóa tầm view và lấy sáng tự nhiên, giúp lưu thông không khí tốt hơn. Chúng em rất chú trọng đến từng chi tiết thiết kế để nâng niu cảm nhận trong từng trải nghiệm của cư dân. Nếu Anh/Chị cần thêm thông tin chi tiết, em sẵn sàng hỗ trợ!


In [8]:
query = "Dự án bên em có gì đặc biệt ?"
# Execute
config = {"configurable": {"thread_id": "abc123"}}
messages = [
    {"role": "user", "content": query}
]
response = await graph.ainvoke({"messages": messages}, config)
print(response["messages"][-1].content)

Dạ, dự án Masteri Grand View có nhiều điểm đặc biệt nổi bật, Anh/Chị có thể tham khảo một số điểm sau:

1. **Vị trí đắc địa**: Dự án tọa lạc trong khu đô thị 117.4ha – The Global City, tại phường An Phú, với vị trí kim cương tại giao điểm đa sắc màu.

2. **Thiết kế hiện đại**: Các căn hộ được thiết kế với ban công rộng rãi, lan can kính cao 1,4m tạo view nhìn thoáng đãng. Tất cả các phòng ngủ đều có cửa kính full trần đến sàn, tối ưu hóa tầm view và ánh sáng tự nhiên.

3. **Tiện ích đỉnh cao**: Dự án thừa hưởng toàn bộ hệ tiện ích cao cấp của The Global City, mang đến cho cư dân một cuộc sống tiện nghi và hiện đại.

4. **Sảnh thang máy đẳng cấp**: Sảnh thang máy được thiết kế lấy cảm hứng từ vẻ đẹp thanh nguyên của chất liệu và đường nét, tạo nên dấu ấn rõ nét và đặc trưng cho dự án.

5. **Loại hình căn hộ khan hiếm**: Dự án sở hữu loại hình căn hộ 4PN cực kỳ khan hiếm trên thị trường, là lựa chọn hàng đầu cho những gia đình lớn.

Nếu Anh/Chị cần thêm thông tin chi tiết, em rất sẵn lòn

In [9]:
import numpy as np
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader("../knowledge_pdf/masterigrandview.pdf")
pages = loader.load()

# 2. Splitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=500,
    chunk_overlap=70,
    length_function=len
)

docs = text_splitter.split_documents(pages)

# Chuyển câu truy vấn thành vector embedding
query_vector = embeddings.embed_query(query)

# Tìm kiếm trong chỉ mục FAISS
D, I = index.search(np.array([query_vector], dtype=np.float32), k=2)
retrieved_docs = []

for idx in I[0]:
    retrieved_docs.append(docs[idx])

serialized = "\n\n".join(
    (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
    for doc in retrieved_docs
)
print(retrieved_docs)

[Document(metadata={'source': '../knowledge_pdf/masterigrandview.pdf', 'page': 15}, page_content='“MỘT SIÊU PHẨM CAO TẦNG \nTHUỘC \nMASTERI COLLECTION\nNGAY TẠI TRUNG TÂM MỚI”'), Document(metadata={'source': '../knowledge_pdf/masterigrandview.pdf', 'page': 43}, page_content='Ban \ncông\n rộng\nrãi\n với\n lan\n can \nkính\n cao\n 1,4m \ntạo\n view \nnhìn\nthoáng\n đãng\nMặt\n bằng\n thiết\n kế\ncho\n phép\n gia\n chủ\n có\nthể\n tối\n ưu\n thi\n công\nthêm\n phần\n đảo\n bếp\nnếu\n muốn\n (kitchen \nisland)\nLogia \nđược\n khéo\n léo\nbố\n trí\n để\n tạo\n ra\n không\ngian\n cho\n sân\n phơi\n, \nhệ\nlam \nkín\n đáo\n che\n chắn\ntăng\n tính\n thẩm\n mỹ\ncho\n cả\n dự\n án\nTất\n cả\n các\n phòng\n ngủ\nđều\n đươc\n thiết\n kế\n hệ\n cửa\nkính\n full \ntrần\n đến\n sàn\n, \ntối\nưu\n hóa\n tầm\n view \nvà\n lấy\nsáng\n tự\n nhiên\n, \nlưu\n thông\nkhông\n khí')]


In [9]:
# Khi tìm kiếm "Tiện ích"